In [99]:
pip install -U -q PyDrive

In [100]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [101]:
import pandas as pd
import numpy as np
import warnings
import time
import csv
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import OneHotEncoder

#from memory_profiler import profile

In [102]:
%matplotlib inline
import matplotlib.pyplot as plt
import csv
from textblob import TextBlob
import pandas
import sklearn
#import cPickle
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
#from sklearn.grid_search import GridSearchCV
#from sklearn.cross_validation import StratifiedKFold, cross_val_score, train_test_split 
from sklearn.tree import DecisionTreeClassifier 
#from sklearn.learning_curve import learning_curve

In [103]:
from sklearn.feature_extraction.text import CountVectorizer

In [104]:
from sklearn.feature_extraction.text import TfidfTransformer

In [105]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [106]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [107]:
import pandas as pd

In [108]:
path = "/content/drive/MyDrive/Data/data.csv"
df_bonus = pd.read_csv(path)

In [109]:
df_bonus

,Unnamed: 0,label,message
0,0,ham,"Go until jurong point, crazy.. Available only ..."
1,1,ham,Ok lar... Joking wif u oni...
2,2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,3,ham,U dun say so early hor... U c already then say...
4,4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...,...
5569,5569,spam,This is the 2nd time we have tried 2 contact u...
5570,5570,ham,Will ü b going to esplanade fr home?
5571,5571,ham,"Pity, * was in mood for that. So...any other s..."
5572,5572,ham,The guy did some bitching but I acted like i'd...


In [110]:
messages = df_bonus

In [111]:
messages.groupby('label').describe()

Unnamed: 0                                                         \
           count         mean          std  min     25%     50%     75%   
label                                                                     
ham       4827.0  2794.346385  1605.244806  0.0  1409.5  2795.0  4185.5   
spam       747.0  2735.797858  1634.851784  2.0  1228.0  2719.0  4151.5   

               
          max  
label          
ham    5573.0  
spam   5569.0

In [112]:
messages['length'] = messages['message'].map(lambda text: len(text))
print(messages.head())

   Unnamed: 0 label                                            message  length
0           0   ham  Go until jurong point, crazy.. Available only ...     111
1           1   ham                      Ok lar... Joking wif u oni...      29
2           2  spam  Free entry in 2 a wkly comp to win FA Cup fina...     155
3           3   ham  U dun say so early hor... U c already then say...      49
4           4   ham  Nah I don't think he goes to usf, he lives aro...      61


In [113]:
from textblob import TextBlob

In [114]:
def split_into_tokens(message):
    message = str(message)  # convert bytes into proper unicode
    return TextBlob(message).words

In [115]:
messages.message.head()

0    Go until jurong point, crazy.. Available only ...
1                        Ok lar... Joking wif u oni...
2    Free entry in 2 a wkly comp to win FA Cup fina...
3    U dun say so early hor... U c already then say...
4    Nah I don't think he goes to usf, he lives aro...
Name: message, dtype: object

In [116]:
import nltk

In [117]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [118]:
def split_into_lemmas(message):
    message = str(message).lower()
    words = TextBlob(message).words
    # for each word, take its "base form" = lemma 
    return [word.lemma for word in words]

messages.message.head().apply(split_into_lemmas)

0    [go, until, jurong, point, crazy, available, o...
1                       [ok, lar, joking, wif, u, oni]
2    [free, entry, in, 2, a, wkly, comp, to, win, f...
3    [u, dun, say, so, early, hor, u, c, already, t...
4    [nah, i, do, n't, think, he, go, to, usf, he, ...
Name: message, dtype: object

In [119]:
bow_transformer = CountVectorizer(analyzer=split_into_lemmas).fit(messages['message'])
print(len(bow_transformer.vocabulary_))

8731


In [120]:
message4 = messages['message'][3]
print(message4)

U dun say so early hor... U c already then say...


In [121]:
bow4 = bow_transformer.transform([message4])
print(bow4)
print(bow4.shape)

  (0, 1152)	1
  (0, 1889)	1
  (0, 2859)	1
  (0, 2889)	1
  (0, 3973)	1
  (0, 6623)	2
  (0, 6993)	1
  (0, 7578)	1
  (0, 7884)	2
(1, 8731)


In [122]:
print(bow_transformer.get_feature_names()[6736])
print(bow_transformer.get_feature_names()[8013])

settled
usc


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [123]:
messages_bow = bow_transformer.transform(messages['message'])
print('sparse matrix shape:', messages_bow.shape)
print('number of non-zeros:', messages_bow.nnz)
print('sparsity: %.2f%%' % (100.0 * messages_bow.nnz / (messages_bow.shape[0] * messages_bow.shape[1])))

sparse matrix shape: (5574, 8731)
number of non-zeros: 80522
sparsity: 0.17%


In [124]:
tfidf_transformer = TfidfTransformer().fit(messages_bow)
tfidf4 = tfidf_transformer.transform(bow4)
print(tfidf4)

  (0, 7884)	0.30462762250720843
  (0, 7578)	0.22407066876175707
  (0, 6993)	0.19104059338743914
  (0, 6623)	0.5237900094551098
  (0, 3973)	0.4567201651112802
  (0, 2889)	0.32993959758075475
  (0, 2859)	0.30393632675119536
  (0, 1889)	0.2468405911204805
  (0, 1152)	0.2745557307081983


In [125]:
print(tfidf_transformer.idf_[bow_transformer.vocabulary_['u']])
print(tfidf_transformer.idf_[bow_transformer.vocabulary_['university']])

2.8438555403215284
8.239753235208429


In [126]:
messages_tfidf = tfidf_transformer.transform(messages_bow)
print(messages_tfidf.shape)

(5574, 8731)


In [127]:
x_train, x_rem, y_train, y_rem = train_test_split(messages['message'],messages['label'], train_size=0.8)

In [128]:
x_valid, x_test, y_valid, y_test = train_test_split(x_rem,y_rem, test_size=0.5)

In [129]:
print(len(x_train), len(x_test), len(x_valid))

4459 558 557


In [159]:
pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=split_into_lemmas)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', MultinomialNB()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

In [160]:
pipeline.fit(x_train,y_train)

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function split_into_lemmas at 0x7ff1713959d0>)),
                ('tfidf', TfidfTransformer()),
                ('classifier', MultinomialNB())])

In [161]:
pipeline.score(x_test, y_test)

0.942652329749104

In [162]:
pipeline.score(x_valid, y_valid)

0.9622980251346499

In [163]:
y_pred = pipeline.predict(x_test)
pred_data = pd.DataFrame()
pred_data['TEST'] = y_test
pred_data['PRED_Y'] = y_pred
pred_data


,TEST,PRED_Y
1339,ham,ham
2225,ham,ham
2083,ham,ham
5439,ham,ham
1440,ham,ham
...,...,...
2584,ham,ham
1266,ham,ham
2171,ham,ham
4849,ham,ham


In [164]:
y_pred = pipeline.predict(x_valid)
pred_data = pd.DataFrame()
pred_data['TEST'] = y_valid
pred_data['PRED_Y'] = y_pred
pred_data

,TEST,PRED_Y
1749,ham,ham
839,spam,spam
1732,ham,ham
1370,ham,ham
3832,ham,ham
...,...,...
559,ham,ham
1477,ham,ham
248,ham,ham
931,ham,ham


In [165]:
from sklearn.tree import DecisionTreeClassifier 

In [166]:
pipeline_decisicionTree = Pipeline([
    ('bow', CountVectorizer(analyzer=split_into_lemmas)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', DecisionTreeClassifier(random_state=42,max_depth = 10)),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

In [167]:
pipeline_decisicionTree.fit(x_train,y_train)

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function split_into_lemmas at 0x7ff1713959d0>)),
                ('tfidf', TfidfTransformer()),
                ('classifier',
                 DecisionTreeClassifier(max_depth=10, random_state=42))])

In [168]:
pipeline_decisicionTree.score(x_test, y_test)

0.953405017921147

In [169]:
pipeline_decisicionTree.score(x_valid, y_valid)

0.9712746858168761

In [170]:
y_pred_tree = pipeline_decisicionTree.predict(x_valid)
pred_data = pd.DataFrame()
pred_data['TEST'] = y_valid
pred_data['PRED_Y'] = y_pred
pred_data

,TEST,PRED_Y
1749,ham,ham
839,spam,spam
1732,ham,ham
1370,ham,ham
3832,ham,ham
...,...,...
559,ham,ham
1477,ham,ham
248,ham,ham
931,ham,ham


In [171]:
confusion_matrix(y_valid, y_pred_tree)

array([[489,   3],
       [ 13,  52]])

In [172]:
print(classification_report(y_valid, y_pred_tree))

              precision    recall  f1-score   support

         ham       0.97      0.99      0.98       492
        spam       0.95      0.80      0.87        65

    accuracy                           0.97       557
   macro avg       0.96      0.90      0.93       557
weighted avg       0.97      0.97      0.97       557



In [173]:
from sklearn.ensemble import RandomForestClassifier

In [174]:
pipeline_randomForest = Pipeline([
    ('bow', CountVectorizer(analyzer=split_into_lemmas)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', RandomForestClassifier(n_estimators = 100,random_state=42,max_depth = 10)),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

In [175]:
pipeline_randomForest.fit(x_train,y_train)

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function split_into_lemmas at 0x7ff1713959d0>)),
                ('tfidf', TfidfTransformer()),
                ('classifier',
                 RandomForestClassifier(max_depth=10, random_state=42))])

In [176]:
pipeline_randomForest.score(x_test, y_test)

0.8781362007168458

In [177]:
pipeline_randomForest.score(x_valid, y_valid)

0.9192100538599641

In [178]:
y_pred_forest = pipeline_randomForest.predict(x_valid)
pred_data = pd.DataFrame()
pred_data['TEST'] = y_valid
pred_data['PRED_Y'] = y_pred
pred_data

,TEST,PRED_Y
1749,ham,ham
839,spam,spam
1732,ham,ham
1370,ham,ham
3832,ham,ham
...,...,...
559,ham,ham
1477,ham,ham
248,ham,ham
931,ham,ham


In [179]:
confusion_matrix(y_valid, y_pred_forest)

array([[492,   0],
       [ 45,  20]])

In [180]:
print(classification_report(y_valid, y_pred_forest))

              precision    recall  f1-score   support

         ham       0.92      1.00      0.96       492
        spam       1.00      0.31      0.47        65

    accuracy                           0.92       557
   macro avg       0.96      0.65      0.71       557
weighted avg       0.93      0.92      0.90       557

